## RNN
Based on the closing price of the Apple stock on a daily basis, use RNN to predict its price. <br>
Data collected from:
https://sg.finance.yahoo.com/quote/AAPL/history?p=AAPL <br>
Train data is from Jan 1st 2016 to Jan 1st 2019 <br>
Test data is from Jan 1st 2019 to Jan 1st 2020 <br>

In [8]:
# Model Training

import torch
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

#Importing training set and only pick "Close" price of the day
dataset_train=pd.read_csv('AAPL.csv')
training_set=dataset_train.iloc[:,4:5].values

#Feature Scaling
from sklearn.preprocessing import MinMaxScaler
sc=MinMaxScaler(feature_range=(0,1))
training_set_scaled=sc.fit_transform(training_set)

#Creating a data structure with 120 timesteps (120 to be optimized)
X_train=[]
y_train=[]
for i in range(120,754):
    X_train.append(training_set_scaled[i-120:i,0])
    y_train.append(training_set_scaled[i,0])
X_train = np.array(X_train)
y_train = np.array(y_train)
X_train = torch.from_numpy(X_train).type(torch.Tensor)
y_train = torch.from_numpy(y_train).type(torch.Tensor)

class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.layer1 = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.layer2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        c = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        h = h.requires_grad_()
        c = c.requires_grad_()
        h = h.detach()
        c = c.detach()
        out, (h_final, c_final) = self.layer1(x, (h, c))
        out = self.layer2(out[:, -1, :])
        return out

#Parameters to be optimized
input_size = 1
hidden_size = 32
num_layers = 2
output_size = 1
num_epochs = 100
model = LSTM(input_size=input_size, hidden_size=hidden_size, output_size=output_size, num_layers=num_layers)
criterion = torch.nn.MSELoss(reduction='mean')
optimiser = torch.optim.Adam(model.parameters(), lr=0.01)

import time
for t in range(num_epochs):
    y_train_pred = model(X_train.unsqueeze(-1))
    loss = criterion(y_train_pred, y_train.unsqueeze(-1))
    print("Epoch", t, "MSE: ", loss.item())
    optimiser.zero_grad()
    loss.backward()
    optimiser.step()


Epoch  0 MSE:  0.28270938992500305
Epoch  1 MSE:  0.14935100078582764
Epoch  2 MSE:  0.0548175610601902
Epoch  3 MSE:  0.22046899795532227
Epoch  4 MSE:  0.0675278827548027
Epoch  5 MSE:  0.05064745247364044
Epoch  6 MSE:  0.06806137412786484
Epoch  7 MSE:  0.0825720727443695
Epoch  8 MSE:  0.08826185017824173
Epoch  9 MSE:  0.08626902848482132
Epoch  10 MSE:  0.07894124835729599
Epoch  11 MSE:  0.06864703446626663
Epoch  12 MSE:  0.05768055096268654
Epoch  13 MSE:  0.04834873229265213
Epoch  14 MSE:  0.042901087552309036
Epoch  15 MSE:  0.04296112060546875
Epoch  16 MSE:  0.047916945070028305
Epoch  17 MSE:  0.05273764207959175
Epoch  18 MSE:  0.05160698667168617
Epoch  19 MSE:  0.045139379799366
Epoch  20 MSE:  0.03788142278790474
Epoch  21 MSE:  0.03290184959769249
Epoch  22 MSE:  0.03046853467822075
Epoch  23 MSE:  0.029172951355576515
Epoch  24 MSE:  0.026896730065345764
Epoch  25 MSE:  0.02111692726612091
Epoch  26 MSE:  0.01126170251518488
Epoch  27 MSE:  0.01363696251064539
Epo

In [9]:
# Send in test data and do evaluation
dataset_test=pd.read_csv('AAPL_test.csv')
testing_set=dataset_test.iloc[:,4:5].values

#Feature Scaling
from sklearn.preprocessing import MinMaxScaler
sc=MinMaxScaler(feature_range=(0,1))
testing_set_scaled=sc.fit_transform(testing_set)

#Creating a data structure with 120 timesteps
X_test=[]
y_test=[]
for i in range(120,754):
    X_test.append(training_set_scaled[i-120:i,0])
    y_test.append(training_set_scaled[i,0])
X_test = np.array(X_test)
y_test = np.array(y_test)
X_test = torch.from_numpy(X_test).type(torch.Tensor)
y_test = torch.from_numpy(y_test).type(torch.Tensor)

y_test_pred = model(X_test.unsqueeze(-1))
loss = criterion(y_test_pred, y_test.unsqueeze(-1))
print("Test MSE: ", loss.item())

Test MSE:  0.0008291492122225463
